# Abrupt Leakage Example

In [ ]:
from IPython.display import display, HTML
display(HTML('<a target="_blank" href="https://colab.research.google.com/github/WaterFutures/EPyT-Flow/blob/main/docs/examples/abrupt_leakage.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>'))

This example demonstrates how add an abrupt leakage to a scenario -- more information can be found in the [documentation](https://epyt-flow.readthedocs.io/en/stable/tut.events.html#leakages).

In [ ]:
%pip install epyt-flow --quiet

In [ ]:
from epyt.epanet import ToolkitConstants
from epyt_flow.data.networks import load_hanoi
from epyt_flow.simulation import ScenarioSimulator, AbruptLeakage
from epyt_flow.utils import to_seconds, plot_timeseries_data

Load the [Hanoi network](https://epyt-flow.readthedocs.io/en/stable/epyt_flow.data.html#epyt_flow.data.networks.load_hanoi) with a default sensor placement:

In [ ]:
hanoi_network_config = load_hanoi(include_default_sensor_placement=True, verbose=False)

Create a new simulation:

In [ ]:
sim = ScenarioSimulator(scenario_config=hanoi_network_config)

Set simulation duration to 7 days:

In [ ]:
sim.set_general_parameters(simulation_duration=to_seconds(days=7))

Note that leakages are only modeled for two flow units. We therefore change the flow units to cubic meter per hour by calling [set_general_parameters()](https://epyt-flow.readthedocs.io/en/stable/epyt_flow.simulation.html#epyt_flow.simulation.scenario_simulator.ScenarioSimulator.set_general_parameters) -- could also be done when loading the network!

In [ ]:
sim.set_general_parameters(flow_units_id=ToolkitConstants.EN_CMH)

Add an abrupt leakage at link/pipe "14" -- the leakage is active for 18 hours and starts at 10 hours after simulation begin -- recall that the time arguments are seconds!

We create an abrupt leakage by creating an instance of the [AbruptLeakage](https://epyt-flow.readthedocs.io/en/stable/epyt_flow.simulation.events.html#epyt_flow.simulation.events.leakages.AbruptLeakage) class and add the leakage to the scenario by calling [add_leakage()](https://epyt-flow.readthedocs.io/en/stable/epyt_flow.simulation.html#epyt_flow.simulation.scenario_simulator.ScenarioSimulator.add_leakage):

In [ ]:
leak = AbruptLeakage(link_id="14", diameter=0.1,
                     start_time=to_seconds(hours=10),
                     end_time=to_seconds(hours=28))

sim.add_leakage(leak)

Run the entire simulation:

In [ ]:
scada_data = sim.run_simulation()

Retrieve and show pressure at node "13" over time:

In [ ]:
pressure_data = scada_data.get_data_pressures(sensor_locations=['13']).T
plot_timeseries_data(pressure_data,
                     x_axis_label="Time (30min steps)", y_axis_label="Pressure in $m$")

Do not forget to close the simulator!

In [ ]:
sim.close()